In [1]:
import torch 
inputs = torch.tensor(
	[[0.43, 0.15, 0.89], # Your    
	[0.55, 0.87, 0.66], # journey (x^2) <<<
	[0.57, 0.85, 0.64], # starts 
	[0.22, 0.58, 0.33], # with    
	[0.77, 0.25, 0.10], # one     
	[0.05, 0.80, 0.55]] # step    
)
inputs 

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [2]:
# Kiểm tra số lượng hàng và cột của ma trận đầu vào 
inputs.shape

torch.Size([6, 3])

In [3]:
# Tính toán các điểm số attention trung gian giữa token truy vấn (query token) và từng token đầu vào
query = inputs[1] # journey (x^2) <<< tensor([0.5500, 0.8700, 0.6600])
attn_scores_2 = torch.empty(inputs.shape[0]) # inputs.shape: [6, 3] >>> inputs.shape[0]: 6
for i, x_i in enumerate(inputs):
	attn_scores_2[i] = torch.dot(x_i, query) # Tích của 2 ma trận cùng cỡ sẽ tạo ra 1 số
print(attn_scores_2) # Đưa số đó vào trong 1 vector [24, 5465, 56, 65, 78, 85]

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [4]:
# Bình thường hóa (normalization) attention_scores_2 (Điểm số attention trung gian)
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
print(">>> Weights Attention:", attn_weights_2)
print(">>> Tổng:", attn_weights_2.sum())

>>> Weights Attention: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
>>> Tổng: tensor(1.)


In [5]:
# Kiểm tra số cột hàng của biến query 
query.shape # >>> torch.Size([3])

torch.Size([3])

In [6]:
# Tạo một vector rỗng có số cột và hàng bằng query.shape (>>> torch.Size([3]))
torch.zeros(query.shape)

tensor([0., 0., 0.])

In [7]:
# Tính toán vector ngữ cảnh 
query = inputs[1] # journey (x^2) <<< tensor([0.5500, 0.8700, 0.6600]), shape: torch.Size([3])
context_vector_2 = torch.zeros(query.shape) # >>> tensor([0., 0., 0.]), shape: torch.Size([3])
for i, x_i in enumerate(inputs):
	context_vector_2 += attn_weights_2[i] * x_i 
print(context_vector_2)

tensor([0.4419, 0.6515, 0.5683])


In [8]:
# Tính toán điểm attention của tất cả inputs 
# Vì dùng for_loop duyệt qua từng phần tử nên 
# ta có thể sử dụng matrix multiplication
attn_scores = torch.empty(6, 6)
for i, x_i in enumerate(inputs):
	for j, x_j in enumerate(inputs):
		attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [9]:
# Matrix multiplication 
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [10]:
# Normalization toàn bộ attention_scores 
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [11]:
# Tính vector_contexts cho toàn bộ inputs 
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [12]:
x_2 = inputs[1] # journey (x^2) <<< tensor([0.5500, 0.8700, 0.6600])
d_in = inputs.shape[1] # d=3, size đầu vào embedding (input_embedding)
d_out = 2 # Size đầu ra output_embedding, d_out=2

In [ ]:
torch.manual_seed(123)
# Khai báo ma trận trọng số (weight_matrices) W_q, W_k, W_v
W_query = 	torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key 	= 	torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = 	torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)